In [1]:
# пробная модель по граф эмбеддингу и последующей классификации

import pickle
import numpy as np

with open("train_info", "rb") as fin:
    data = pickle.load(fin)
    X_train, y_train, X_train_skel_features = data["data"], data["labels"], data["skel_features"]

with open("test_info", "rb") as fin:
    data = pickle.load(fin)
    X_test, y_test, X_test_skel_features = data["data"], data["labels"], data["skel_features"]

In [2]:
print(len(y_test))

10000


In [3]:
#немного уменьшаем корпуса
train_size = 10000
test_size = 1000
import numpy, collections
def unison_shuffled_copies(a, b, size):
    assert len(a) == len(b)
    p = numpy.random.permutation(len(a))
    return a[p][:size], b[p][:size]

y_train, X_train_skel_features = unison_shuffled_copies(np.asarray(y_train), np.asarray(X_train_skel_features), train_size)
y_test, X_test_skel_features = unison_shuffled_copies(np.asarray(y_test), np.asarray(X_test_skel_features), test_size)
print(collections.Counter(y_train))
print(collections.Counter(y_test))

Counter({1: 1108, 3: 1052, 7: 1038, 6: 1011, 2: 988, 0: 983, 9: 971, 4: 963, 5: 953, 8: 933})
Counter({1: 123, 7: 107, 2: 106, 6: 104, 5: 99, 9: 98, 3: 93, 8: 93, 0: 92, 4: 85})


In [4]:
def my_get_edges_ged_rad(skel_features):
    edges_x = []
    edges_y = []
    degs = []
    rads = []
    for i, d in enumerate(skel_features):
        if i % 4 == 0:
            edges_x.append(d)
        if i % 4 == 1:
            edges_y.append(d)
        if i % 4 == 2:
            #degs.append([[edges_x[-1], edges_y[-1]], d])
            deg = d
        if i % 4 == 3:
            #rads.append([[edges_x[-1], edges_y[-1]], d])
            rad = d
            degs.append([(edges_x[-1], edges_y[-1]), deg, rad])
    return degs

In [5]:
# достаем признаки скелетов
skel_feats = []
for skel in X_train_skel_features:
    skel_feats.append(my_get_edges_ged_rad(skel))
    
skel_feats_test = []
for skel in X_test_skel_features:
    skel_feats_test.append(my_get_edges_ged_rad(skel))

In [6]:
# в качестве вершин графа в постановке Node2Vec должны быть строки, поэтому индексируем вершины,
# заданные своими координатами. Для индексирования сортируем вершины по высоте (координате y)
node_dicts = []
node_dicts_test = []
def takeSecond(elem):
    return elem[1]

for skel_feat in skel_feats:
    node_dict = {}
    nodes = [t[0] for t in skel_feat]
    for i,j in enumerate(sorted(list(set(nodes)), key=takeSecond)):
        node_dict[j] = str(i)
    node_dicts.append(node_dict)
    
for skel_feat in skel_feats_test:
    node_dict = {}
    nodes = [t[0] for t in skel_feat]
    for i,j in enumerate(sorted(list(set(nodes)), key=takeSecond)):
        node_dict[j] = str(i)
    node_dicts_test.append(node_dict)

In [7]:
import networkx as nx

In [8]:
#конструируем графы в представлении NetrworkX
graphs = []
for j in range(len(skel_feats)):
    g = nx.Graph()
    for i in range(len(skel_feats[j])):
        if i % 2 == 0:
        #att = {'deg':degs_[i][1]}
            g.add_edge(u_of_edge=node_dicts[j][skel_feats[j][i][0]], v_of_edge=node_dicts[j][skel_feats[j][i+1][0]], weight=np.sqrt((skel_feats[j][i][0][0])**2+(skel_feats[j][i][0][1])**2))
    graphs.append(g)
    
graphs_test = []
for j in range(len(skel_feats_test)):
    g = nx.Graph()
    for i in range(len(skel_feats_test[j])):
        if i % 2 == 0:
        #att = {'deg':degs_[i][1]}
            g.add_edge(u_of_edge=node_dicts_test[j][skel_feats_test[j][i][0]], v_of_edge=node_dicts_test[j][skel_feats_test[j][i+1][0]], weight=np.sqrt((skel_feats_test[j][i][0][0])**2+(skel_feats_test[j][i][0][1])**2))
    graphs_test.append(g)

In [9]:
print(len(graphs), len(graphs_test))

10000 1000


In [10]:
import networkx as nx
from node2vec import Node2Vec

#обучаем эмбеддинги графов из обучающей выборки
embedding_matrixes = []
for i in range(len(graphs)):
# Precompute probabilities and generate walks
    if i % 100 == 0:
        print(i)
    node2vec = Node2Vec(graph=graphs[i], dimensions=10, walk_length=15,
                        num_walks=10, workers=4, weight_key='weight', quiet=True)

# Embed nodes
    model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

    embedding_matrix = np.array([])
    for t in range(len(node_dicts[i])):    
        embedding_matrix = np.append(embedding_matrix, np.asarray(model.wv[str(t)]))
    del model
    embedding_matrixes.append(embedding_matrix)

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900


In [13]:
print(len(embedding_matrixes_test))

1000


In [12]:
#обучаем эмбеддинги графов из обучающей выборки
embedding_matrixes_test = []
for i in range(len(graphs_test)):
# Precompute probabilities and generate walks
    if i % 100 == 0:
        print(i)
    node2vec = Node2Vec(graph=graphs_test[i], dimensions=10, walk_length=15,
                        num_walks=20, workers=4, weight_key='weight', quiet=True)

# Embed nodes
    model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `diemnsions` and `workers` are automatically passed (from the Node2Vec constructor)

    embedding_matrix = np.array([])
    for t in range(len(node_dicts_test[i])):    
        embedding_matrix = np.append(embedding_matrix, np.asarray(model.wv[str(t)]))
    del model
    embedding_matrixes_test.append(embedding_matrix)

0
100
200
300
400
500
600
700
800
900


In [14]:
sizes = np.asarray([len(d) for d in node_dicts])
max_size = 10*sizes.max()
print(max_size)

sizes_test = np.asarray([len(d) for d in node_dicts_test])
max_size_test = 10*sizes.max()
print(max_size_test)
assert max_size < 700 and max_size_test < 700


600
600


In [15]:
max_size = 600

In [16]:
#приводим все описания графов к константной длине
for i in range(len(embedding_matrixes)):
    
    if len(embedding_matrixes[i]) < max_size:
        embedding_matrixes[i] = np.append(embedding_matrixes[i], np.zeros(max_size - len(embedding_matrixes[i])))
for i in range(len(embedding_matrixes_test)):
    
    if len(embedding_matrixes_test[i]) < max_size:
        embedding_matrixes_test[i] = np.append(embedding_matrixes_test[i], np.zeros(max_size - len(embedding_matrixes_test[i])))

In [17]:
print(len(embedding_matrixes[7]), len(embedding_matrixes_test[7]))

600 600


In [28]:
# описываем перцептрон
import tensorflow as tf
n_hidden = 64
xavier_init = tf.contrib.layers.xavier_initializer(seed=12)

weights = {
    'h1': tf.Variable(xavier_init([max_size, n_hidden])),
    'h2': tf.Variable(xavier_init([n_hidden, n_hidden])),
    'h3': tf.Variable(xavier_init([n_hidden, n_hidden])),
    'h4': tf.Variable(xavier_init([n_hidden, 10]))
}

biases = {
    'b1': tf.Variable(tf.zeros([n_hidden])),
    'b2': tf.Variable(tf.zeros([n_hidden])),
    'b3': tf.Variable(tf.zeros([n_hidden])),
    'b4': tf.Variable(tf.zeros([10]))
}

def neural_net(input):
    output = tf.nn.leaky_relu(tf.add(tf.matmul(input, weights['h1']), biases['b1']))
    output = tf.nn.leaky_relu(tf.add(tf.matmul(output, weights['h2']), biases['b2']))
    output = tf.nn.leaky_relu(tf.add(tf.matmul(output, weights['h3']), biases['b3']))
    output = tf.add(tf.matmul(output, weights['h4']), biases['b4'])
    return output

X = tf.placeholder("float", [None, max_size])
Y = tf.placeholder("float", [None, 10])


# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=0.005)
train_op = optimizer.minimize(loss_op)


pred = tf.nn.softmax(logits)
lab = tf.argmax(pred, 1)
# Apply softmax to logits
correct_prediction = tf.equal(tf.argmax(pred, 1), tf.argmax(Y, 1))
    # Calculate accuracy
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

In [37]:
from sklearn.metrics import f1_score
def one_hot_y(a):
    out =[]
    for c in a:
        result = np.zeros(10, dtype = 'int')
        result[c] = 1
        out.append(result)
    return out

batch_size = 100
init = tf.global_variables_initializer()
with tf.Session() as sess:
    sess.run(init)
    #
    for i in range(int(len(y_train)/batch_size)):
        loss, _ = sess.run((loss_op, train_op), feed_dict = {X:embedding_matrixes[i*batch_size:(i+1)*batch_size], Y:one_hot_y(y_train[i*batch_size:(i+1)*batch_size])})
        
        if i % 10 == 0:
            print(loss)
    accs = np.array([])
    learned_labels_ = np.array([], dtype = 'int')
    for i in range(int(len(y_test)/batch_size)):
        #acc = sess.run(accuracy, feed_dict = {X:embedding_matrixes_test[i*batch_size:(i+1)*batch_size], Y:one_hot_y(y_test[i*batch_size:(i+1)*batch_size])})
        #accs = np.append(accs, acc)
   #print(np.mean(accs))
        learned_labels = sess.run(lab, feed_dict = {X:embedding_matrixes_test[i*batch_size:(i+1)*batch_size], Y:one_hot_y(y_test[i*batch_size:(i+1)*batch_size])})
        #print(len(learned_labels))
        learned_labels_ = np.append(learned_labels_, learned_labels)
    #print(learned_labels_)
    score = f1_score(y_true=y_test,y_pred=learned_labels_, average=None)
    f1_dict = dict(zip([0,1,2,3,4,5,6,7,8,9], score))
    print('F1 classification score',f1_dict)
    

2.3663654
2.0915818
1.9650061
2.032762
1.7501487
1.8280786
1.8860668
1.8505545
1.8155171
1.9980762
F1 classification score {0: 0.3537414965986394, 1: 0.71356783919598, 2: 0.0, 3: 0.11650485436893203, 4: 0.023255813953488372, 5: 0.2626582278481012, 6: 0.0, 7: 0.3922829581993569, 8: 0.2, 9: 0.0}


/home/boeing/.local/venvs/tfenv/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [34]:
# как мы видим, модель получилась неадекватной :()